In [2]:
import pandas as pd
import numpy as np
import os
import glob
import datetime


In [3]:
def remove_decimals(df, column_name):
    """
    Removes Decimal points from columns in data frames.
    """
    try:
        df[column_name] = df[column_name].astype(str)
        m = df[column_name].str.contains('\d+')
        df.loc[m, column_name] = df.loc[m, column_name].apply(lambda x: x.split('.')[0])
        df.loc[m, column_name] = df.loc[m, column_name].astype('int64')
    except ValueError:
        pass

In [4]:
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 100)
pd.set_option('display.min_rows', 100)
pd.set_option('display.expand_frame_repr', True)

In [5]:
df_fi = pd.read_excel(r'C:\Users\sappidi.reddy\Downloads\Sample MIAG 3\FI.xlsx', dtype={'XBLNR':str, 'BUKRS': str,'BELNR':str,'Document_type':str,'Company code':str,"store_or_dc":str,'ZBD1T':'Int64','suppl_no':str,'LIFNR':str,'REDAT':str,'GEBRF':str,'GSMWB':str,'WEARS': str,'GSMWF':str,'WENUM':str,'RGDAT':str, 'AUGBL':str,'ABGST':str,'VORGN':str,'GJAHR':str,'DEBNOTNO':str,'BUZEI': str})

In [6]:
df_ic_mmsic = pd.read_excel(r'C:\Users\sappidi.reddy\Downloads\Sample MIAG 3\MMSIC.xlsx',dtype={'RENR': str, 'BELNR':str,'AUFNR':str,'WEDAT':str,'ZBD1T':'Int64','LIFNR':str,'REDAT':str,'GEBRF':str,'GSMWB':str,'WEARS': str,'GSMWF':str,'WENUM':str,'RGDAT':str,'ABGST':str,'VORGN':str,'GJAHR':str,'VORGN':str})

df_ic_sis =  pd.read_excel(r'C:\Users\sappidi.reddy\Downloads\Sample MIAG 3\SIS.xlsx',dtype={'RENR': str, 'BELNR':str,'AUFNR':str,'WEDAT':str,'ZBD1T':'Int64','LIFNR':str,'REDAT':str,'GEBRF':str,'GSMWB':str,'WEARS': str,'GSMWF':str,'WENUM':str,'RGDAT':str,'ABGST':str,'VORGN':str,'GJAHR':str,'VORGN':str})

df_ic = pd.concat([df_ic_mmsic,df_ic_sis], axis = 0, ignore_index = True)

In [7]:
df_ic.to_csv(r'C:\Users\sappidi.reddy\Downloads\Sample MIAG 3\combined_ic.csv', index=False)

In [8]:
sftp_df = pd.read_csv(r'C:\Users\sappidi.reddy\Downloads\Sample MIAG 3\miag.35.258800.20241111.942.csv',dtype = {"Store":str,"Supplier number (MIAG)":str,"Remittance advice number":str,"Supplier number (Sales Line)":str,"Document number":str, "Invoice number":str})

In [9]:
df_fi = df_fi[df_fi['Document_type'] != 'PM']

In [10]:
df_fi['XBLNR'] = df_fi['XBLNR'].astype(str).apply(lambda x: x.zfill(10) if x.isnumeric() else x)

In [11]:
df_fi.shape

(16673, 35)

In [12]:
df_ic.shape

(80648, 17)

In [13]:
sftp_df.shape

(8290, 18)

In [14]:
doc_to_type_ic = dict(zip(df_ic['BELNR'], df_ic['RENR']))

    # Fill missing values in FI_df['XBLNR'] using the mapping and FI_df['BELNR']
df_fi['XBLNR'] = df_fi['XBLNR'].fillna(df_fi['BELNR'].map(doc_to_type_ic))

In [15]:
doc_to_type_fi = dict(zip(df_fi['XBLNR'], df_fi['BELNR']))

df_ic['BELNR'] = df_ic['BELNR'].str.strip().replace('', np.nan)
    # Fill missing values in CI_df['BELNR'] using the mapping and CI_df['RENR']
df_ic['BELNR'] = df_ic['BELNR'].fillna(df_ic['RENR'].map(doc_to_type_fi))

In [16]:
grouped_fi = df_fi.groupby(['BELNR', 'suppl_no'], as_index=False)

# Step 2: Populate values for ZFBDT, ZBD1T, ZBD2T
def populate_values(group):
    group['ZFBDT'] = group['ZFBDT'].fillna(method='ffill').fillna(method='bfill')
    group['ZBD1T'] = group['ZBD1T'].fillna(method='ffill').fillna(method='bfill')
    group['ZBD2T'] = group['ZBD2T'].fillna(method='ffill').fillna(method='bfill')
    return group

df_fi_filled = grouped_fi.apply(populate_values)

# Reset index to avoid ambiguity
df_fi_filled.reset_index(drop=True, inplace=True)

# Step 3: Find the record with the highest absolute TDC, then lowest BUZEI in case of ties
def select_record(group):
    max_abs_tdc = group['Amount_in_local_currency'].abs().max()
    filtered = group[group['Amount_in_local_currency'].abs() == max_abs_tdc]
    return filtered.loc[filtered['BUZEI'].idxmin()]

df_fi = df_fi_filled.groupby(['BELNR', 'suppl_no'], as_index=False).apply(select_record).reset_index(drop=True)



C:\Users\sappidi.reddy\AppData\Local\Temp\1\ipykernel_4352\4053083448.py:5: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  group['ZFBDT'] = group['ZFBDT'].fillna(method='ffill').fillna(method='bfill')
C:\Users\sappidi.reddy\AppData\Local\Temp\1\ipykernel_4352\4053083448.py:6: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  group['ZBD1T'] = group['ZBD1T'].fillna(method='ffill').fillna(method='bfill')
C:\Users\sappidi.reddy\AppData\Local\Temp\1\ipykernel_4352\4053083448.py:7: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  group['ZBD2T'] = group['ZBD2T'].fillna(method='ffill').fillna(method='bfill')
C:\Users\sappidi.reddy\AppData\Local\Temp\1\ipykernel_4352\4053083448.py:10: DeprecationWarning: DataFrameGroupBy.apply operated on t

In [17]:
df_fi.shape

(4865, 35)

In [18]:
def get_max_populated_row(group):
    # Count non-null values for each row
    non_null_counts = group.notnull().sum(axis=1)
    # Get the index of the row with the maximum count
    max_index = non_null_counts.idxmax()
    return group.loc[max_index]

In [19]:
df_ic = df_ic.groupby('RENR').apply(get_max_populated_row).reset_index(drop=True)

C:\Users\sappidi.reddy\AppData\Local\Temp\1\ipykernel_4352\4076789276.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_ic = df_ic.groupby('RENR').apply(get_max_populated_row).reset_index(drop=True)


In [20]:
df_ic.shape

(2753, 17)

In [21]:
df_fi.shape

(4865, 35)

In [22]:
sftp_df.shape

(8290, 18)

## standarized the data

In [23]:
#preprocess ic/FI/sftp document numbers. make the length 10 or above

df_ic['LIFNR'] = df_ic['LIFNR'].apply(
    lambda x: '10000' + str(x) if not pd.isna(x) and len(str(x)) < 10 else str(x) if not pd.isna(x) else '0' * 10
)

df_fi['suppl_no'] = df_fi['suppl_no'].apply(
    lambda x: '10000' + str(x) if not pd.isna(x) and len(str(x)) < 10 else str(x) if not pd.isna(x) else '0' * 10
)

df_ic['BELNR'] = df_ic['BELNR'].apply(
     lambda x: '0' * (10 - len(str(x))) + str(x) if not pd.isna(x) else '0' * 10)

df_fi['BELNR'] = df_fi['BELNR'].apply(
     lambda x: '0' * (10 - len(str(x))) + str(x) if not pd.isna(x) else '0' * 10)

sftp_df['Document number'] = sftp_df['Document number'].apply(
     lambda x: '0' * (10 - len(str(x))) + str(x) if not pd.isna(x) else '0' * 10)
     


In [24]:
def transform_number(num):
    if pd.isna(num):  # Check for NaN values
        return np.nan   # Return NaN if the input is NaN
    num_str = str(num)  # Convert to string

    # If the number already starts with '10000', return it as is
    if num_str.startswith('10000'):
        return num_str
    
    stripped_num = num_str.lstrip('0')  # Remove leading zeros
    final_num = stripped_num.zfill(6)  # Ensure it has at least 6 digits
    return '1000' + final_num  # Prepend '10000'

# Apply the function to the relevant column

df_ic['LIFNR'] = df_ic['LIFNR'].apply(transform_number)

In [25]:
#PREPROCESS DATE TIME. make all %d.%m.%Y

for col in df_ic.select_dtypes(include=['datetime64[ns]', 'datetime']):
    df_ic[col] = df_ic[col].dt.strftime('%d.%m.%Y')

for col in df_fi.select_dtypes(include=['datetime64[ns]', 'datetime']):
    df_fi[col] = df_fi[col].dt.strftime('%d.%m.%Y')
    

In [26]:
df_fi['BLDAT']

0       02.10.2023
1       02.10.2023
2       02.10.2023
3       02.10.2023
4       02.10.2023
5       02.10.2023
6       02.10.2023
7       02.10.2023
8       02.10.2023
9       02.10.2023
10      02.10.2023
11      02.10.2023
12      02.10.2023
13      02.10.2023
14      02.10.2023
15      02.10.2023
16      02.10.2023
17      02.10.2023
18      02.10.2023
19      02.10.2023
20      02.10.2023
21      02.10.2023
22      02.10.2023
23      02.10.2023
24      02.10.2023
25      02.10.2023
26      02.10.2023
27      02.10.2023
28      02.10.2023
29      02.10.2023
30      02.10.2023
31      02.10.2023
32      02.10.2023
33      02.10.2023
34      02.10.2023
35      02.10.2023
36      02.10.2023
37      02.10.2023
38      02.10.2023
39      02.10.2023
40      02.10.2023
41      02.10.2023
42      02.10.2023
43      02.10.2023
44      02.10.2023
45      02.10.2023
46      02.10.2023
47      02.10.2023
48      02.10.2023
49      02.10.2023
           ...    
4815    19.08.2024
4816    19.0

In [27]:
#convert string types into %d.%m.%Y. IC frame
for col in df_ic.select_dtypes(include=['object']):
    try:
        # Convert to datetime, assuming the format is 'YYYYMMDD'
        df_ic[col] = pd.to_datetime(df_ic[col], format='%Y%m%d')
        # Optionally, you can convert it to DMY format
        df_ic[col] = df_ic[col].dt.strftime('%d.%m.%Y')
    except ValueError:
        # If conversion fails, just continue
        pass

In [28]:
#convert string types into %d.%m.%Y. IC frame
for col in df_fi.select_dtypes(include=['object']):
    try:
        # Convert to datetime, assuming the format is 'YYYYMMDD'
        df_fi[col] = pd.to_datetime(df_fi[col], format='%Y%m%d')
        # Optionally, you can convert it to DMY format
        df_fi[col] = df_fi[col].dt.strftime('%d.%m.%Y')
    except ValueError:
        # If conversion fails, just continue
        pass

In [171]:
df_ic.to_csv(r"C:\Users\sappidi.reddy\Downloads\Sample MIAG 3\ic_data_cleaned.csv", index=False)

In [172]:
df_fi.to_csv(r"C:\Users\sappidi.reddy\Downloads\Sample MIAG 3\fi_data_cleaned.csv", index=False)

In [29]:
df_fi['Amount_in_local_currency'] = df_fi['Amount_in_local_currency'] * -1

In [30]:
#merge FI and IC

merge_fi_ic = df_fi.merge(df_ic, left_on=[ "BELNR","XBLNR"], right_on=[
         "BELNR","RENR"], how="outer")


In [175]:
merge_fi_ic.to_csv(r"C:\Users\sappidi.reddy\Downloads\Sample MIAG 3\merge1_check.csv", index=False)

In [31]:
print(merge_fi_ic.shape)

(5204, 51)


In [32]:
#check lengths and dtyes of merged frame
measurer = np.vectorize(len)
res = measurer(merge_fi_ic.values.astype(str)).max(axis=0)
res = dict(zip(merge_fi_ic, measurer(merge_fi_ic.values.astype(str)).max(axis=0)))
print(res)

{'MANDT': np.int64(5), 'Document_type': np.int64(3), 'document_type_desc': np.int64(20), 'GJAHR_x': np.int64(4), 'BUKRS': np.int64(4), 'GSBER': np.int64(3), 'PRCTR': np.int64(10), 'store_or_dc': np.int64(4), 'KOSTL': np.int64(12), 'month_in_fin_year': np.int64(4), 'BELNR': np.int64(10), 'XBLNR': np.int64(16), 'AUGBL': np.int64(10), 'AUGDT': np.int64(10), 'ZFBDT': np.int64(10), 'ZBD1T': np.int64(4), 'ZBD2T': np.int64(3), 'NETDT': np.int64(3), 'BUZEI': np.int64(3), 'altkt': np.int64(12), 'hkont': np.int64(12), 'suppl_no': np.int64(10), 'BLDAT': np.int64(10), 'BUDAT': np.int64(10), 'CPUDT': np.int64(10), 'partition_date': np.int64(10), 'dana_ingestion_date': np.int64(10), 'shkzg': np.int64(3), 'Amount_in_local_currency': np.int64(11), 'Amount_in_document_currency': np.int64(11), 'Tax_in_local_currency': np.int64(3), 'Tax_in_document_currency': np.int64(3), 'WAERS_x': np.int64(3), 'Batch_Input_session_name': np.int64(12), 'sgtxt': np.int64(40), 'LIFNR': np.int64(10), 'RENR': np.int64(16), 

In [33]:
### sftp with merged, ic and fi data
merged_df = merge_fi_ic.merge(
        sftp_df, left_on=["BELNR", "XBLNR"],
        right_on=["Document number", "Invoice number"],
        how="outer")

In [34]:
merged_df[['BLDAT', 'RGDAT']]

,BLDAT,RGDAT
0,NaN,19.10.2024
1,NaN,24.09.2024
2,NaN,22.10.2024
3,NaN,19.10.2024
4,NaN,01.10.2024
5,NaN,19.10.2024
6,NaN,09.10.2024
7,NaN,31.10.2024
8,NaN,19.10.2024
9,NaN,28.10.2024


In [35]:
df_fi['BLDAT']

0       02.10.2023
1       02.10.2023
2       02.10.2023
3       02.10.2023
4       02.10.2023
5       02.10.2023
6       02.10.2023
7       02.10.2023
8       02.10.2023
9       02.10.2023
10      02.10.2023
11      02.10.2023
12      02.10.2023
13      02.10.2023
14      02.10.2023
15      02.10.2023
16      02.10.2023
17      02.10.2023
18      02.10.2023
19      02.10.2023
20      02.10.2023
21      02.10.2023
22      02.10.2023
23      02.10.2023
24      02.10.2023
25      02.10.2023
26      02.10.2023
27      02.10.2023
28      02.10.2023
29      02.10.2023
30      02.10.2023
31      02.10.2023
32      02.10.2023
33      02.10.2023
34      02.10.2023
35      02.10.2023
36      02.10.2023
37      02.10.2023
38      02.10.2023
39      02.10.2023
40      02.10.2023
41      02.10.2023
42      02.10.2023
43      02.10.2023
44      02.10.2023
45      02.10.2023
46      02.10.2023
47      02.10.2023
48      02.10.2023
49      02.10.2023
           ...    
4815    19.08.2024
4816    19.0

In [181]:
merged_df.to_csv(r"C:\Users\sappidi.reddy\Downloads\Sample MIAG 3\merge2_check_sftp.csv", index=False)

PermissionError: [Errno 13] Permission denied: 'C:\\Users\\sappidi.reddy\\Downloads\\Sample MIAG 3\\merge2_check_sftp.csv'

In [36]:
merged_df.shape

(9627, 69)

In [37]:
# Split the ARKTX column by '#'
split_columns = merged_df['ARKTX'].str.split('#', expand=True)

# Extract Business Year and Line Item No., ensuring they stay as integers if ARKTX is not blank
# merged_df['Business Year SFTP'] = split_columns[3].apply(lambda x: int(x) if pd.notnull(x) and x.isdigit() else pd.NA).astype('Int64')
# merged_df['Line Item No. SFTP'] = split_columns[4].apply(lambda x: int(x) if pd.notnull(x) and x.isdigit() else pd.NA).astype('Int64')

merged_df['Business Year SFTP'] = split_columns[3].apply(
    lambda x: str(int(x)) if pd.notnull(x) and x.isdigit() else pd.NA
).astype('string')

merged_df['Line Item No. SFTP'] = split_columns[4].apply(
    lambda x: str(int(x)) if pd.notnull(x) and x.isdigit() else pd.NA
).astype('string')




In [38]:
merged_df.shape

(9627, 71)

In [39]:
merged_df['Doc no. combined'] = merged_df['BELNR'].fillna(merged_df['Document number'])

In [40]:
merged_df.shape

(9627, 72)

In [44]:
pd.set_option("display.max_rows", 50)

## populate ARKTX

In [187]:
# Assuming merged_df is your DataFrame
merged_df["year"] = merged_df.BLDAT.apply(lambda x: x.split('.')[2] if isinstance(x, str) and len(x.split('.')) > 2 else None)
merged_df["month"] = merged_df.BLDAT.apply(lambda x: x.split('.')[1] if isinstance(x, str) and len(x.split('.')) > 1 else None)
merged_df["day"] = merged_df.BLDAT.apply(lambda x: x.split('.')[0] if isinstance(x, str) and len(x.split('.')) > 0 else None)

mask_empty_ARKTX = merged_df["ARKTX"].isna()

# Apply the logic only on those rows
merged_df.loc[mask_empty_ARKTX, "ARKTX"] = merged_df[mask_empty_ARKTX].apply(
    lambda row: (
        f"{row['Document_type']}#{row['BELNR']}#{row['year']}{row['month']}{row['day']}#{row['year']}#00{row['BUZEI']}"
        if all(pd.notna([row['Document_type'], row['BELNR'], row['year'], row['month'], row['day'], row['BUZEI']]))
        else np.nan
    ),
    axis=1
)

## net due calculations

In [188]:
merged_df['ZFBDT'] = pd.to_datetime(merged_df['ZFBDT'], format='%d.%m.%Y')

merged_df['ZBD1T'] = pd.to_numeric(merged_df['ZBD1T'], errors='coerce')
merged_df['ZBD2T'] = pd.to_numeric(merged_df['ZBD2T'], errors='coerce')

merged_df['NET_DUE_DATE'] = merged_df.apply(
    lambda row: row['ZFBDT'] + pd.Timedelta(days=row['ZBD1T'] if pd.notna(row['ZBD1T']) else (row['ZBD2T'] if pd.notna(row['ZBD2T']) else 0)),
    axis=1
)
merged_df['NET_DUE_DATE'] = merged_df['NET_DUE_DATE'].dt.strftime('%d.%m.%Y')

## Invoice status assignment

In [189]:
ic_tran_status_df = pd.read_csv(r"C:\Users\sappidi.reddy\Downloads\Sample MIAG 3\ic_transaction_status_R.csv", encoding='windows-1252', dtype={'TRANSACTION STATUS (ABGST)': 'str', 'LBL - \nTRANSACTION STATUS (ABGST)':'str', 'VIPA \nTRANSACTION STATUS (ABGST)':'str',"360 invoice status proposal (25.4.)":'str'})
ic_tran_status_df = ic_tran_status_df[["TRANSACTION STATUS (ABGST)", "LBL - \nTRANSACTION STATUS (ABGST)", "VIPA \nTRANSACTION STATUS (ABGST)", "360 invoice status proposal (25.4.)"]]
ic_tran_status_df["ABGST"] = ic_tran_status_df["TRANSACTION STATUS (ABGST)"].fillna(ic_tran_status_df['LBL - \nTRANSACTION STATUS (ABGST)']).fillna(ic_tran_status_df["VIPA \nTRANSACTION STATUS (ABGST)"])
ic_tran_status_df = ic_tran_status_df[["ABGST", "360 invoice status proposal (25.4.)"]]
abgst_status_dict = ic_tran_status_df.set_index('ABGST')['360 invoice status proposal (25.4.)'].to_dict()
len(abgst_status_dict)

197

In [190]:
# Step 1: Populate 'Inv Stat' with "cleared-MIAG" for rows where 'RAN' is present
merged_df.loc[merged_df['Remittance advice number'].notna(), 'INVOICE_STATUS'] = "cleared-MIAG"

# Step 2: Populate 'Inv Stat' with "cleared-FI" for rows where 'Inv Stat' is null and 'AUGBL' is present
merged_df.loc[merged_df['INVOICE_STATUS'].isna() & merged_df['AUGBL'].notna(), 'INVOICE_STATUS'] = "cleared-FI"

# Step 3: Populate 'Inv Stat' with "Invoice Approval completed" for rows where 'Inv Stat' is null and 'BLDAT' is present
merged_df.loc[merged_df['INVOICE_STATUS'].isna() & merged_df['BLDAT'].notna(), 'INVOICE_STATUS'] = "Invoice approval completed"

# Step 4: Populate 'Inv Stat' based on mapping from abgst_status_dict for rows where 'Inv Stat' is null and 'ABGST' is present
merged_df.loc[merged_df['INVOICE_STATUS'].isna() & merged_df['ABGST'].notna(), 'INVOICE_STATUS'] = merged_df['ABGST'].map(abgst_status_dict)

# Step 5: Populate remaining 'Inv Stat' as "In progress" where 'Inv Stat' is still null
merged_df['INVOICE_STATUS'].fillna("In progress", inplace=True)

C:\Users\sappidi.reddy\AppData\Local\Temp\1\ipykernel_1820\592722656.py:14: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merged_df['INVOICE_STATUS'].fillna("In progress", inplace=True)


In [50]:
merged_df.shape

(9627, 77)

## GR Invoice Implmentation

In [51]:
# Separate rows with Doc no. combined having all 0's
df_all_zeros = merged_df[merged_df['Doc no. combined'] == '0000000000']

# Exclude rows with Doc no. combined having all 0's for further grouping
df_non_zero = merged_df[merged_df['Doc no. combined'] != '0000000000']

# Group by Doc no. combined
grouped = df_non_zero.groupby('Doc no. combined')

# Filter groups based on their lengths
df_group_len_2 = grouped.filter(lambda x: len(x) == 2)
df_group_len_1 = grouped.filter(lambda x: len(x) == 1)
df_group_len_3 = grouped.filter(lambda x: len(x) > 2)

# Now we have the three dataframes
# df_all_zeros: rows with Doc no. combined having all 0's
# df_group_len_1: groups with length 1
# df_group_len_2: groups with length 2
# df_group_len_3: groups with greater than length 2


In [52]:
print("All zeros : ", len(df_all_zeros))
print("Length 1 : ", len(df_group_len_1))
print("Length 2 : ", len(df_group_len_2))
print("Length 3 : ", len(df_group_len_3))

All zeros :  51
Length 1 :  8582
Length 2 :  856
Length 3 :  138


In [53]:
df_group_len_2.to_csv(r'C:\Users\sappidi.reddy\Downloads\Sample MIAG 3\gr_invs_len_2.csv', index=False)

In [54]:
df_group_len_3.to_csv(r'C:\Users\sappidi.reddy\Downloads\Sample MIAG 3\gr_invs_len_3.csv', index=False)

In [55]:
# Group the dataframe by 'Doc no. combined'
grouped = df_group_len_2.groupby('Doc no. combined')

# Initialize empty dataframes
both_cleared_or_one_progress = pd.DataFrame()
gr_invoice_records = pd.DataFrame()
not_cleared_miag_records = pd.DataFrame()

# Iterate over each group
for doc_no, group in grouped:
    statuses = group['INVOICE_STATUS'].tolist()
    
    # Check conditions for both_cleared_or_one_progress
    if statuses == ['cleared-MIAG', 'cleared-MIAG'] or \
       ('cleared-MIAG' in statuses and 'In progress' in statuses):
        both_cleared_or_one_progress = pd.concat([both_cleared_or_one_progress, group])
    # Check conditions for gr_invoice_records
    elif 'cleared-MIAG' in statuses and any(status in ['cleared-FI', 'Invoice approval completed'] for status in statuses):
        gr_invoice_records = pd.concat([gr_invoice_records, group])
    elif all(status != 'cleared-MIAG' for status in statuses):
        not_cleared_miag_records = pd.concat([not_cleared_miag_records, group])

# Reset index for the new DataFrames if needed
both_cleared_or_one_progress.reset_index(drop=True, inplace=True)
gr_invoice_records.reset_index(drop=True, inplace=True)
not_cleared_miag_records.reset_index(drop=True, inplace=True)

# both_cleared_or_one_progress and gr_invoice_records now contain the desired groups


In [56]:
print("Length 2 : ", len(df_group_len_2))
print("Both cleared or one progress : ", len(both_cleared_or_one_progress))
print("GR Invoice Records : ", len(gr_invoice_records))
print("not_cleared_miag_records ", len(not_cleared_miag_records))

Length 2 :  856
Both cleared or one progress :  46
GR Invoice Records :  810
not_cleared_miag_records  0


In [57]:
both_cleared_or_one_progress

,MANDT,Document_type,document_type_desc,GJAHR_x,BUKRS,GSBER,PRCTR,store_or_dc,KOSTL,month_in_fin_year,BELNR,XBLNR,AUGBL,AUGDT,ZFBDT,ZBD1T,ZBD2T,NETDT,BUZEI,altkt,hkont,suppl_no,BLDAT,BUDAT,CPUDT,...,Supplier number (Sales Line),Supplier number (MIAG),Supplier name,VAT number,Document number,Invoice number,Document type,Document date,Remittance advice number,Value date,Currency,Gross amount,Description,Contract indicator,Store,Company code,ARKTX,Business Year SFTP,Line Item No. SFTP,Doc no. combined,year,month,day,NET_DUE_DATE,INVOICE_STATUS
0,100.0,WE,Goods receiving (inv,2024,3142,NaN,31421037.0,1037,NaN,1.0,0810027271,A262023000004676,NaN,NaN,2023-10-16,45.0,NaN,NaN,1,2.521100e+09,1.332110e+09,1000020870,16.10.2023,16.10.2023,24.10.2023,...,1000020870,3500007956,FRİTO LAY GIDA SAN. VE TİC.A .Ş.,9.000051e+09,0810027271,A262023000004676,WE,16.10.2023,03516445326,08.12.2023,TRY,10115.00,cleared,No MIAG contract,1037,3142.0,WE#0810027271#20231016#2024#001,2024,1,0810027271,2023,10,16,30.11.2023,cleared-MIAG
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1000015545,3500003274,ERGIDA GIDA SAN.TİC.AŞ.,3.550043e+09,0810027271,EG12022000000712,WE,12.10.2022,03515575340,11.01.2023,TRY,180278.41,cleared,MIAG contract,6092,3142.0,WE#0810027271#20221012#2023#001,2023,1,0810027271,None,None,None,NaN,cleared-MIAG
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1000020870,3500007956,FRİTO LAY GIDA SAN. VE TİC.A .Ş.,9.000051e+09,0810083178,A032022000024315,WE,08.12.2022,03515657956,10.02.2023,TRY,1292.27,cleared,No MIAG contract,1022,3142.0,WE#0810083178#20221208#2023#001,2023,1,0810083178,None,None,None,NaN,cleared-MIAG
3,100.0,WE,Goods receiving (inv,2024,3142,NaN,31421028.0,1028,NaN,3.0,0810083178,A532023000011952,NaN,NaN,2023-11-30,45.0,NaN,NaN,1,2.521100e+09,1.332110e+09,1000020870,30.11.2023,01.12.2023,06.12.2023,...,1000020870,3500007956,FRİTO LAY GIDA SAN. VE TİC.A .Ş.,9.000051e+09,0810083178,A532023000011952,WE,30.11.2023,03516538949,16.01.2024,TRY,13635.40,cleared,No MIAG contract,1028,3142.0,WE#0810083178#20231130#2024#001,2024,1,0810083178,2023,11,30,14.01.2024,cleared-MIAG
4,100.0,WE,Goods receiving (inv,2024,3142,NaN,31421032.0,1032,NaN,3.0,0810093709,A032023000013745,NaN,NaN,2023-11-30,45.0,NaN,NaN,1,2.521100e+09,1.332110e+09,1000020870,30.11.2023,01.12.2023,14.12.2023,...,1000020870,3500007956,FRİTO LAY GIDA SAN. VE TİC.A .Ş.,9.000051e+09,0810093709,A032023000013745,WE,30.11.2023,03516538949,16.01.2024,TRY,13205.30,cleared,No MIAG contract,1032,3142.0,WE#0810093709#20231130#2024#001,2024,1,0810093709,2023,11,30,14.01.2024,cleared-MIAG
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1000020870,3500007956,FRİTO LAY GIDA SAN. VE TİC.A .Ş.,9.000051e+09,0810093709,A082022000014224,WE,22.12.2022,03515664028,14.02.2023,TRY,666.59,cleared,No MIAG contract,1023,3142.0,WE#0810093709#20221222#2023#001,2023,1,0810093709,None,None,None,NaN,cleared-MIAG
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1000020870,3500007956,FRİTO LAY GIDA SAN. VE TİC.A .Ş.,9.000051e+09,0810095881,A062022000003196,WE,21.11.2022,03515657956,10.02.2023,TRY,2776.44,cleared,No MIAG contract,1044,3142.0,WE#0810095881#20221121#2023#001,2023,1,0810095881,None,None,None,NaN,cleared-MIAG
7,100.0,WE,Goods receiving (inv,2024,3142,NaN,31421031.0,1031,NaN,3.0,0810095881,A642023000001782,NaN,NaN,2023-12-06,45.0,NaN,NaN,1,2.521100e+09,1.332110e+09,1000020870,06.12.2023,06.12.2023,15.12.2023,...,1000020870,3500007956,FRİTO LAY GIDA SAN. VE TİC.A .Ş.,9.000051e+09,0810095881,A642023000001782,WE,06.12.2023,03516555589,23.01.2024,TRY,7352.21,cleared,No MIAG contract,1031,3142.0,WE#0810095881#20231206#2024#001,2024,1,0810095881,2023,12,06,20.01.2024,cleared-MIAG
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [58]:
df_group_len_2

,MANDT,Document_type,document_type_desc,GJAHR_x,BUKRS,GSBER,PRCTR,store_or_dc,KOSTL,month_in_fin_year,BELNR,XBLNR,AUGBL,AUGDT,ZFBDT,ZBD1T,ZBD2T,NETDT,BUZEI,altkt,hkont,suppl_no,BLDAT,BUDAT,CPUDT,...,Supplier number (Sales Line),Supplier number (MIAG),Supplier name,VAT number,Document number,Invoice number,Document type,Document date,Remittance advice number,Value date,Currency,Gross amount,Description,Contract indicator,Store,Company code,ARKTX,Business Year SFTP,Line Item No. SFTP,Doc no. combined,year,month,day,NET_DUE_DATE,INVOICE_STATUS
719,100.0,WE,Goods receiving (inv,2024,3142,NaN,31421037.0,1037,NaN,1.0,0810027271,A262023000004676,NaN,NaN,2023-10-16,45.0,NaN,NaN,1,2.521100e+09,1.332110e+09,1000020870,16.10.2023,16.10.2023,24.10.2023,...,1000020870,3500007956,FRİTO LAY GIDA SAN. VE TİC.A .Ş.,9.000051e+09,0810027271,A262023000004676,WE,16.10.2023,03516445326,08.12.2023,TRY,10115.00,cleared,No MIAG contract,1037,3142.0,WE#0810027271#20231016#2024#001,2024,1,0810027271,2023,10,16,30.11.2023,cleared-MIAG
720,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1000015545,3500003274,ERGIDA GIDA SAN.TİC.AŞ.,3.550043e+09,0810027271,EG12022000000712,WE,12.10.2022,03515575340,11.01.2023,TRY,180278.41,cleared,MIAG contract,6092,3142.0,WE#0810027271#20221012#2023#001,2023,1,0810027271,None,None,None,NaN,cleared-MIAG
804,100.0,WE,Goods receiving (inv,2024,3142,NaN,31421023.0,1023,NaN,1.0,0810031586,A082023000006299,NaN,NaN,2023-10-23,45.0,NaN,NaN,1,2.521100e+09,1.332110e+09,1000020870,23.10.2023,23.10.2023,27.10.2023,...,1000020870,3500007956,FRİTO LAY GIDA SAN. VE TİC.A .Ş.,9.000051e+09,0810031586,A082023000006299,WE,23.10.2023,03516445326,08.12.2023,TRY,2158.52,cleared,No MIAG contract,1023,3142.0,WE#0810031586#20231023#2024#001,2024,1,0810031586,2023,10,23,07.12.2023,cleared-MIAG
805,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0810031586,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,0810031586,None,None,None,NaN,Invoice approval completed
807,100.0,WE,Goods receiving (inv,2024,3142,NaN,31421024.0,1024,NaN,1.0,0810031651,A262023000004675,NaN,NaN,2023-10-16,45.0,NaN,NaN,1,2.521100e+09,1.332110e+09,1000020870,16.10.2023,16.10.2023,27.10.2023,...,1000020870,3500007956,FRİTO LAY GIDA SAN. VE TİC.A .Ş.,9.000051e+09,0810031651,A262023000004675,WE,16.10.2023,03516445326,08.12.2023,TRY,12283.27,cleared,No MIAG contract,1024,3142.0,WE#0810031651#20231016#2024#001,2024,1,0810031651,2023,10,16,30.11.2023,cleared-MIAG
808,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0810031651,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,0810031651,None,None,None,NaN,Invoice approval completed
873,100.0,WE,Goods receiving (inv,2024,3142,NaN,31421032.0,1032,NaN,1.0,0810035970,A032023000012231,NaN,NaN,2023-10-21,45.0,NaN,NaN,1,2.521100e+09,1.332110e+09,1000020870,21.10.2023,21.10.2023,31.10.2023,...,1000020870,3500007956,FRİTO LAY GIDA SAN. VE TİC.A .Ş.,9.000051e+09,0810035970,A032023000012231,WE,21.10.2023,03516445326,08.12.2023,TRY,9937.77,cleared,No MIAG contract,1032,3142.0,WE#0810035970#20231021#2024#001,2024,1,0810035970,2023,10,21,05.12.2023,cleared-MIAG
874,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0810035970,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,0810035970,None,None,None,NaN,Invoice approval completed
1520,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1000020870,3500007956,FRİTO LAY GIDA SAN. VE TİC.A .Ş.,9.000051e+09,0810083178,A032022000024315,WE,08.12.2022,03515657956,10.02.2023,TRY,1292.27,cleared,No MIAG contract,1022,3142.0,WE#0810083178#20221208#2023#001,2023,1,0810083178,None,None,None,NaN,cleared-MIAG
1521,100.0,WE,Goods receiving (inv,2024,3142,NaN,31421028.0,1028,N

In [59]:
# Group by 'Doc no. combined'
grouped = gr_invoice_records.groupby('Doc no. combined')

# Initialize lists to store the rows based on INVOICE_STATUS
cleared_records = []
other_records = []

# Iterate through each group
for name, group in grouped:
    # Separate rows based on 'INVOICE_STATUS' value
    cleared_record = group[group['INVOICE_STATUS'] == 'cleared-MIAG']
    other_record = group[group['INVOICE_STATUS'] != 'cleared-MIAG']
    
    # Append to the lists if the record exists
    if not cleared_record.empty:
        cleared_records.append(cleared_record.iloc[0])  # Assuming there's only one 'cleared-MIAG' record per group
    if not other_record.empty:
        other_records.append(other_record.iloc[0])      # Assuming there's only one non-'cleared-MIAG' record per group

# Convert lists to DataFrames if needed
cleared_df = pd.DataFrame(cleared_records)
other_df = pd.DataFrame(other_records)


In [60]:
print("GR Invoice Records : ", len(gr_invoice_records))
print("Cleared df : ", len(cleared_df))
print("Non Cleared df : ", len(other_df))

GR Invoice Records :  810
Cleared df :  405
Non Cleared df :  405


## Validation starts

In [61]:
cleared_Df_sorted = cleared_df.sort_values('Doc no. combined').reset_index(drop=True)
other_df_sorted = other_df.sort_values('Doc no. combined').reset_index(drop=True)

# Combine the relevant columns for side-by-side display
combined = pd.DataFrame({
    'Doc no. combined': cleared_Df_sorted['Doc no. combined'],  # Assuming Doc no. matches
    'Diff amount': cleared_Df_sorted['Gross amount'] - other_df_sorted['Amount_in_local_currency'],
})

In [62]:
len(combined[abs(combined['Diff amount'])>0.0001])

32

## Validation ends

In [63]:
cleared_df

,MANDT,Document_type,document_type_desc,GJAHR_x,BUKRS,GSBER,PRCTR,store_or_dc,KOSTL,month_in_fin_year,BELNR,XBLNR,AUGBL,AUGDT,ZFBDT,ZBD1T,ZBD2T,NETDT,BUZEI,altkt,hkont,suppl_no,BLDAT,BUDAT,CPUDT,...,Supplier number (Sales Line),Supplier number (MIAG),Supplier name,VAT number,Document number,Invoice number,Document type,Document date,Remittance advice number,Value date,Currency,Gross amount,Description,Contract indicator,Store,Company code,ARKTX,Business Year SFTP,Line Item No. SFTP,Doc no. combined,year,month,day,NET_DUE_DATE,INVOICE_STATUS
0,100.0,WE,Goods receiving (inv,2024,3142,NaN,31421023.0,1023,NaN,1.0,0810031586,A082023000006299,NaN,NaN,2023-10-23,45.0,NaN,NaN,1,2.521100e+09,1.332110e+09,1000020870,23.10.2023,23.10.2023,27.10.2023,...,1000020870,3500007956,FRİTO LAY GIDA SAN. VE TİC.A .Ş.,9.000051e+09,0810031586,A082023000006299,WE,23.10.2023,03516445326,08.12.2023,TRY,2158.52,cleared,No MIAG contract,1023,3142.0,WE#0810031586#20231023#2024#001,2024,1,0810031586,2023,10,23,07.12.2023,cleared-MIAG
2,100.0,WE,Goods receiving (inv,2024,3142,NaN,31421024.0,1024,NaN,1.0,0810031651,A262023000004675,NaN,NaN,2023-10-16,45.0,NaN,NaN,1,2.521100e+09,1.332110e+09,1000020870,16.10.2023,16.10.2023,27.10.2023,...,1000020870,3500007956,FRİTO LAY GIDA SAN. VE TİC.A .Ş.,9.000051e+09,0810031651,A262023000004675,WE,16.10.2023,03516445326,08.12.2023,TRY,12283.27,cleared,No MIAG contract,1024,3142.0,WE#0810031651#20231016#2024#001,2024,1,0810031651,2023,10,16,30.11.2023,cleared-MIAG
4,100.0,WE,Goods receiving (inv,2024,3142,NaN,31421032.0,1032,NaN,1.0,0810035970,A032023000012231,NaN,NaN,2023-10-21,45.0,NaN,NaN,1,2.521100e+09,1.332110e+09,1000020870,21.10.2023,21.10.2023,31.10.2023,...,1000020870,3500007956,FRİTO LAY GIDA SAN. VE TİC.A .Ş.,9.000051e+09,0810035970,A032023000012231,WE,21.10.2023,03516445326,08.12.2023,TRY,9937.77,cleared,No MIAG contract,1032,3142.0,WE#0810035970#20231021#2024#001,2024,1,0810035970,2023,10,21,05.12.2023,cleared-MIAG
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1000020870,3500007956,FRİTO LAY GIDA SAN. VE TİC.A .Ş.,9.000051e+09,0810185908,A172023000001064,WE,15.02.2023,03515861130,28.04.2023,TRY,916.16,cleared,No MIAG contract,1021,3142.0,WE#0810185908#20230215#2023#001,2023,1,0810185908,None,None,None,NaN,cleared-MIAG
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1000020870,3500007956,FRİTO LAY GIDA SAN. VE TİC.A .Ş.,9.000051e+09,0845002564,ICA2023000040086,WB,19.10.2023,03516394518,21.11.2023,TRY,-3554.83,cleared,No MIAG contract,1027,3142.0,WB#0845002564#20231019#2024#001,2024,1,0845002564,None,None,None,NaN,cleared-MIAG
11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1000015545,3500003274,ERGIDA GIDA SAN.TİC.AŞ.,3.550043e+09,0845002621,ICA2023000040235,WB,19.10.2023,03516523827,10.01.2024,TRY,-35352.10,cleared,MIAG contract,6092,3142.0,WB#0845002621#20231019#2024#001,2024,1,0845002621,None,None,None,NaN,cleared-MIAG
13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1000015545,3500003274,ERGIDA GIDA SAN.TİC.AŞ.,3.550043e+09,0845003263,ICA2023000041238,WB,25.10.2023,03516523827,10.01.2024,TRY,-11721.41,cleared,MIAG contract,6092,3142.0,WB#0845003263#20231025#2024#001,2024,1,0845003263,None,None,None,NaN,cleared-MIAG
15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1000015545,3500003274,ERGIDA GIDA SAN.TİC.AŞ.,3.550043e+09,0845003666,ICA2023000041757,WB,26.10.2023,03516361301,08.11.2023,TRY,-24939.01,cleared,MIAG contract,6092,3142.0,WB#0845003666#20231026#2024#001,2024,1,0845003666,None,None,None,NaN,cleared-MIAG
16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1000010074,3500000073,DİVERSEY KİMYA SAN VE TİC A.Ş.,2.950016e+09,0845004592,0024099101,WB,03.11.2023,0

In [64]:
other_df

,MANDT,Document_type,document_type_desc,GJAHR_x,BUKRS,GSBER,PRCTR,store_or_dc,KOSTL,month_in_fin_year,BELNR,XBLNR,AUGBL,AUGDT,ZFBDT,ZBD1T,ZBD2T,NETDT,BUZEI,altkt,hkont,suppl_no,BLDAT,BUDAT,CPUDT,...,Supplier number (Sales Line),Supplier number (MIAG),Supplier name,VAT number,Document number,Invoice number,Document type,Document date,Remittance advice number,Value date,Currency,Gross amount,Description,Contract indicator,Store,Company code,ARKTX,Business Year SFTP,Line Item No. SFTP,Doc no. combined,year,month,day,NET_DUE_DATE,INVOICE_STATUS
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0810031586,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,0810031586,None,None,None,NaN,Invoice approval completed
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0810031651,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,0810031651,None,None,None,NaN,Invoice approval completed
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0810035970,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,0810035970,None,None,None,NaN,Invoice approval completed
7,100.0,WE,Goods receiving (inv,2024,3142,NaN,31426091.0,6091,NaN,5.0,0810185908,PI02024000010274,NaN,NaN,2024-02-20,NaN,NaN,NaN,1,2.521100e+09,1.332110e+09,1000010080,20.02.2024,20.02.2024,27.02.2024,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,WE#0810185908#20240220#2024#001,<NA>,<NA>,0810185908,2024,02,20,20.02.2024,Invoice approval completed
8,100.0,WB,"Debit, claims proces",2024,3142,NaN,31421027.0,1027,NaN,1.0,0845002564,0024094768,NaN,NaN,2023-10-07,45.0,NaN,NaN,1,2.521100e+09,1.332110e+09,1000020870,19.10.2023,19.10.2023,19.10.2023,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,WB#0845002564#20231019#2023#001,<NA>,<NA>,0845002564,2023,10,19,21.11.2023,Invoice approval completed
10,100.0,WB,"Debit, claims proces",2024,3142,NaN,31426092.0,6092,NaN,1.0,0845002621,0024094919,NaN,NaN,2023-09-28,75.0,NaN,NaN,1,2.521100e+09,1.332110e+09,1000015545,19.10.2023,19.10.2023,19.10.2023,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,WB#0845002621#20231019#2023#001,<NA>,<NA>,0845002621,2023,10,19,12.12.2023,Invoice approval completed
12,100.0,WB,"Debit, claims proces",2024,3142,NaN,31426092.0,6092,NaN,1.0,0845003263,0024095996,NaN,NaN,2023-10-05,75.0,NaN,NaN,1,2.521100e+09,1.332110e+09,1000015545,25.10.2023,25.10.2023,25.10.2023,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,WB#0845003263#20231025#2023#001,<NA>,<NA>,0845003263,2023,10,25,19.12.2023,Invoice approval completed
14,100.0,WB,"Debit, claims proces",2024,3142,NaN,31426092.0,6092,NaN,1.0,0845003666,0024096544,NaN,NaN,2023-10-18,NaN,NaN,NaN,1,2.521100e+09,1.332110e+09,1000015545,26.10.2023,26.10.2023,27.10.2023,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,WB#0845003666#20231026#2023#001,<NA>,<NA>,0845003666,2023,10,26,18.10.2023,Invoice approval completed
17,100.0,WB,"Debit, claims proces",2024,3142,NaN,31421021.0,1021,NaN,2.0,0845004592,ICA2023000044000,4200006251,08.11.2023,2022-12-13,NaN,NaN,NaN,1,2.521100e+09,1.332110e+09,1000010074,03.11.2023,03.11.2023,06.11.2023,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,WB#0845004592#20231103#2023#001,<NA>,<NA>,0845004592,2023,11,03,13.12.2022,cleared-FI
19,100.0,WB,"Debit, claims proces",2024,3142,NaN,31421039.0,1039,NaN,2.0,0845004640,ICA2023000044049,4200006251,08.11.2023,2022-12-13,NaN,NaN,NaN,1,2.521100e+09,1.332110e+09,1000010074,03.11.2023,03.11.2023,06.11.2023,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,WB#0845004640#20231103#2023#001,<NA>,<NA>,0845004640,2023,11,03,13.12.2022,cleared-FI


In [65]:
# Ensure correct data types in both dataframes for comparison
other_df['BUZEI'] = other_df['BUZEI'].fillna(0).astype('string')
other_df['GJAHR_x'] = other_df['GJAHR_x'].astype('string')
other_df['Document_type'] = other_df['Document_type'].astype(str)
cleared_df['Document type'] = cleared_df['Document type'].astype(str)
cleared_df['Gross amount'] = cleared_df['Gross amount'].astype(float)
other_df['Amount_in_local_currency'] = other_df['Amount_in_local_currency'].astype(float)

# List to store Doc no. combined values that fail validation
failed_doc_nos = []

# Loop through each row in cleared_df and perform validations
for index, row1 in cleared_df.iterrows():
    # Get the corresponding row in other_df based on 'Doc no. combined'
    row2 = other_df[other_df['Doc no. combined'] == row1['Doc no. combined']]

    # Proceed only if there is exactly one matching row in other_df
    if len(row2) == 1:
        row2 = row2.iloc[0]

        # Perform validation checks
        if (
            row1['Line Item No. SFTP'] == row2['BUZEI'] and
            row1['Document type'] == row2['Document_type'] and
            row1['Business Year SFTP'] == row2['GJAHR_x'] and
            row1['Gross amount'] == row2['Amount_in_local_currency']
        ):
            cleared_df.at[index, 'NET_DUE_DATE'] = row2['NET_DUE_DATE']
            
            # Check if 'Store' column in row1 is not empty
            if pd.notnull(row1['Store']) and row1['Store'] != '':
                # Directly remove row2 from other_df
                other_df = other_df.drop(row2.name)
            else:
                # Copy 'store_or_dc' value from row2 to 'Store' column in row1
                cleared_df.at[index, 'Store'] = row2['store_or_dc']
                # Remove row2 from other_df
                other_df = other_df.drop(row2.name)
        else:
            # If validation fails, add the 'Doc no. combined' to the failed list
            failed_doc_nos.append(row1['Doc no. combined'])
    # else:
    #     # If no match or multiple matches, add to failed list
    #     failed_doc_nos.append(row1['Doc no. combined'])

# Output the updated cleared_df, other_df, and the failed list
cleared_df.reset_index(drop=True, inplace=True)
other_df.reset_index(drop=True, inplace=True)

In [66]:
cleared_df['Line Item No. SFTP'].isna().sum()

np.int64(0)

In [67]:
print("Cleared df : ", len(cleared_df))
print("Non Cleared df : ", len(other_df))
print("Failed docs : ", len(failed_doc_nos))

Cleared df :  405
Non Cleared df :  35
Failed docs :  35


In [68]:
failed_doc_nos

['0810031586',
 '0810031651',
 '0810035970',
 '0810185908',
 '1102686259',
 '3360000841',
 '3360000842',
 '3360000843',
 '3360004958',
 '3360005911',
 '3360005913',
 '3360007847',
 '3360011881',
 '3360012784',
 '3360012786',
 '3360012789',
 '3360013819',
 '3360014733',
 '3360014934',
 '3360015692',
 '3360016877',
 '3360016878',
 '3360017128',
 '3360018034',
 '3360018941',
 '3360019139',
 '3360021189',
 '3360021190',
 '3360022285',
 '3360022685',
 '3360023273',
 '3360023275',
 '3360025222',
 '3360026729',
 '4420000642']

In [69]:
import pandas as pd

# Split the DataFrame into two: len_3_cleared_df and len_3_other_df
len_3_cleared_df = df_group_len_3[df_group_len_3['INVOICE_STATUS'] == 'cleared-MIAG'].copy()
len_3_other_df = df_group_len_3[df_group_len_3['INVOICE_STATUS'] != 'cleared-MIAG'].copy()

# Convert data types as specified
len_3_other_df['BUZEI'] = len_3_other_df['BUZEI'].astype('string')
len_3_other_df['GJAHR_x'] = len_3_other_df['GJAHR_x'].astype('string')
len_3_other_df['Document_type'] = len_3_other_df['Document_type'].astype(str)
len_3_cleared_df['Document type'] = len_3_cleared_df['Document type'].astype(str)
len_3_cleared_df['Gross amount'] = len_3_cleared_df['Gross amount'].astype(float)
len_3_other_df['Amount_in_local_currency'] = len_3_other_df['Amount_in_local_currency'].astype(float)

# Iterate over rows in len_3_other_df
rows_to_delete = []
for index_other, row2 in len_3_other_df.iterrows():
    # Find matching row in len_3_cleared_df
    match = len_3_cleared_df[
        (len_3_cleared_df['Line Item No. SFTP'] == row2['BUZEI']) &
        (len_3_cleared_df['Document type'] == row2['Document_type']) &
        (len_3_cleared_df['Business Year SFTP'] == row2['GJAHR_x']) &
        (len_3_cleared_df['Gross amount'] == row2['Amount_in_local_currency']) &
        (len_3_cleared_df['Doc no. combined'] == row2['Doc no. combined'])
    ]
    
    if not match.empty:
        # Take the first matched row (assuming only one match is expected)
        row1 = match.iloc[0]

        len_3_cleared_df.loc[match.index[0], 'NET_DUE_DATE'] = row2['NET_DUE_DATE']
        
        # Check the Store column in row1
        if pd.isna(row1['Store']):
            # Check the store_or_dc column in row2
            if not pd.isna(row2.get('store_or_dc')):
                len_3_cleared_df.loc[match.index[0], 'Store'] = row2['store_or_dc']
        
        # Mark the row2 for deletion
        rows_to_delete.append(index_other)

# Delete rows from len_3_other_df that were processed
len_3_other_df.drop(index=rows_to_delete, inplace=True)

# Resulting DataFrames: len_3_cleared_df and len_3_other_df


In [70]:
print("Length 3 : ", len(df_group_len_3))
print("len_3_cleared_df : ", len(len_3_cleared_df))
print("len_3_other_df : ", len(len_3_other_df))

Length 3 :  138
len_3_cleared_df :  128
len_3_other_df :  0


In [71]:
merged_df = pd.concat([len_3_cleared_df, len_3_other_df, cleared_df, other_df, both_cleared_or_one_progress, df_group_len_1, df_all_zeros], ignore_index=True)

# Optionally, reset the index if you want a clean index
merged_df.reset_index(drop=True, inplace=True)

In [72]:
len(merged_df)

9247

In [73]:
merged_df.shape

(9247, 77)

## creating the final file

In [74]:
loadfile_df = pd.DataFrame([])

In [75]:
#when the data is present both the system and not in sftp

In [76]:
doc_to_type = dict(zip(merged_df['Supplier number (Sales Line)'],
                           merged_df['Supplier number (MIAG)']))
doc_to_type

{'1000020870': '3500007956',
 '1000015545': '3500003274',
 '1000010080': '3500000079',
 '1000012359': '3500000317',
 '1000010074': '3500000073',
 nan: nan}

In [77]:
len(doc_to_type)

6

# COMPANY CODE

In [78]:
company_code = 3142
loadfile_df['COMPANY_CODE'] = company_code

In [79]:
loadfile_df[['COMPANY_CODE']].isna().sum()

COMPANY_CODE    0
dtype: int64

# Supplier number (Sales Line)

In [80]:
merged_df['Supplier number (Sales Line)'].isna().sum()

np.int64(957)

In [81]:
loadfile_df['SUPPLIER_NO'] = merged_df['Supplier number (Sales Line)'].fillna(
    merged_df['suppl_no']
).fillna(
    merged_df['LIFNR']
)

In [82]:
loadfile_df['SUPPLIER_NO'].isna().sum()

np.int64(0)

# MIAG_SUPPLIER_NO

In [83]:
loadfile_df['MIAG_SUPPLIER_NO'] = merged_df['Supplier number (MIAG)']

In [84]:
loadfile_df['MIAG_SUPPLIER_NO'].isna().sum()

np.int64(957)

In [85]:
loadfile_df['MIAG_SUPPLIER_NO'] = loadfile_df['MIAG_SUPPLIER_NO'].fillna(loadfile_df['SUPPLIER_NO'].map(doc_to_type))

In [86]:
loadfile_df['MIAG_SUPPLIER_NO'].isna().sum()


np.int64(0)

# ORDER_NO

In [87]:
loadfile_df['ORDER_NO'] = merged_df['AUFNR']

In [88]:
loadfile_df['ORDER_NO'].isna().sum()

np.int64(6494)

# DOC Type

In [89]:
loadfile_df['DOC_TYPE'] = merged_df['Document type'].where(merged_df['Document type'].notna(), merged_df['Document_type'])

In [90]:
loadfile_df['DOC_TYPE'].isna().sum()

np.int64(336)

# Invoice number

In [91]:
loadfile_df['INVOICE_NO'] = merged_df['Invoice number'].where(merged_df['Invoice number'].notna(), merged_df['XBLNR'])

In [92]:
loadfile_df['INVOICE_NO'].isna().sum()

np.int64(476)

In [93]:
loadfile_df['INVOICE_NO'] = merged_df['Invoice number'].fillna(
    merged_df['XBLNR']
).fillna(
    merged_df['RENR']
)

In [94]:
loadfile_df['INVOICE_NO'].isna().sum()

np.int64(137)

# INVOICE_DATE

In [95]:
loadfile_df['INVOICE_DATE']	=  merged_df['Document date'].fillna(merged_df['REDAT'])

# DELIVERY_NOTE_NO

In [96]:
loadfile_df['DELIVERY_NOTE_NO'] = merged_df['LFSNR']

# TOTAL_AMT_DC

In [97]:
loadfile_df['TOTAL_AMT_DC'] = merged_df['Gross amount'].fillna(
    merged_df['GEBRF']
).fillna(
    merged_df['Amount_in_local_currency']
)

In [98]:
loadfile_df['TOTAL_AMT_DC'].isna().sum()

np.int64(0)

# TOTAL_VAT_DC

In [99]:
loadfile_df['TOTAL_VAT_DC']	= merged_df['GSMWF']

# CURRENCY

In [100]:
company_code = 3142
loadfile_df['COMPANY_CODE'] = company_code

In [101]:
loadfile_df['COMPANY_CODE'].isna().sum()

np.int64(0)

In [102]:
country_currency_dict = {3142: 'TRY'}

In [103]:
loadfile_df['CURRENCY'] = loadfile_df['COMPANY_CODE'].map(country_currency_dict)

In [104]:
loadfile_df['CURRENCY'].isna().sum()

np.int64(0)

# OTHER COLUMNS

In [105]:
condition = (loadfile_df['DOC_TYPE'] == 'MV')

loadfile_df['PRE_FINANCE_DATE'] = np.where(condition, merged_df['Value date'], '')

loadfile_df['GOODS_RECEIPT_NO']	= merged_df['WENUM']

loadfile_df['GOODS_RECEIPT_DATE'] = merged_df['WEDAT']

loadfile_df['INVOICE_ENTRY_DATE'] = merged_df['RGDAT'].where(merged_df['RGDAT'].notna(), merged_df['BLDAT'])

loadfile_df['INVOICE_STATUS'] = merged_df['INVOICE_STATUS']

loadfile_df['INVOICE_STATUS_INTERNAL'] = merged_df['ABGST']

loadfile_df['NET_DUE_DATE'] = merged_df['NET_DUE_DATE']

loadfile_df['DEBIT_NOTE_NO'] = merged_df['DEBNOTNO']

loadfile_df['REMITTANCE_ADVICE_NO'] = np.where(
    merged_df['INVOICE_STATUS'] == 'cleared-MIAG', 
    merged_df['Remittance advice number'], 
    '')

loadfile_df['CLEARING_DATE'] =  merged_df['Value date'].where(merged_df['Value date'].notna(), merged_df['AUGDT'])

loadfile_df['DOCUMENT_NO']	= merged_df['BELNR'].where(merged_df['BELNR'].notna(), merged_df['Document number'])

# store_no

In [106]:
loadfile_df['STORE_NO'] =  merged_df['Store'].fillna(merged_df['store_or_dc'])

In [107]:
loadfile_df['STORE_NO'].isna().sum()

np.int64(551)

# ARKTX

In [108]:
loadfile_df['ARKTX'] =  merged_df['ARKTX']

# Remaining columns

In [109]:
current_date = datetime.date.today()
formatted_current_date = current_date.strftime("%d.%m.%Y")
loadfile_df['MATCHING_DATE'] = formatted_current_date
loadfile_df['MATCH_STATUS'] = 'No Matching Requested'
loadfile_df['SYNC_DATE']	= formatted_current_date
loadfile_df['SYNC_STATUS'] = '1'
loadfile_df= loadfile_df.fillna('')

In [110]:
loadfile_df['INVOICE_NO'].replace('nan', '', inplace=True)

C:\Users\sappidi.reddy\AppData\Local\Temp\1\ipykernel_1820\104409768.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  loadfile_df['INVOICE_NO'].replace('nan', '', inplace=True)


In [111]:
loadfile_df

,COMPANY_CODE,SUPPLIER_NO,MIAG_SUPPLIER_NO,ORDER_NO,DOC_TYPE,INVOICE_NO,INVOICE_DATE,DELIVERY_NOTE_NO,TOTAL_AMT_DC,TOTAL_VAT_DC,CURRENCY,PRE_FINANCE_DATE,GOODS_RECEIPT_NO,GOODS_RECEIPT_DATE,INVOICE_ENTRY_DATE,INVOICE_STATUS,INVOICE_STATUS_INTERNAL,NET_DUE_DATE,DEBIT_NOTE_NO,REMITTANCE_ADVICE_NO,CLEARING_DATE,DOCUMENT_NO,STORE_NO,ARKTX,MATCHING_DATE,MATCH_STATUS,SYNC_DATE,SYNC_STATUS
0,3142,1000020870,3500007956,,WN,0024114494,25.12.2023,,-371.87,,TRY,,,,,cleared-MIAG,,25.12.2023,,03516496053,28.12.2023,0847512221,1036,WN#0847512221#20231225#2024#001,18.11.2024,No Matching Requested,18.11.2024,1
1,3142,1000020870,3500007956,,WN,AAB2023000000433,02.01.2023,,-1205.81,,TRY,,,,,cleared-MIAG,,,,03515657956,10.02.2023,0847512221,1050,WN#0847512221#20230102#2023#001,18.11.2024,No Matching Requested,18.11.2024,1
2,3142,1000020870,3500007956,,WN,0023136065,03.03.2023,,-98.56,,TRY,,,,,cleared-MIAG,,,,03515727984,09.03.2023,0847522702,1032,WN#0847522702#20230303#2023#001,18.11.2024,No Matching Requested,18.11.2024,1
3,3142,1000020870,3500007956,,WN,ICA2024000015972,11.03.2024,,-4259.58,,TRY,,,,,cleared-MIAG,,10.03.2024,,03516701300,19.03.2024,0847522702,1051,WN#0847522702#20240311#2024#001,18.11.2024,No Matching Requested,18.11.2024,1
4,3142,1000020870,3500007956,,WN,0023141599,20.03.2023,,-172.01,,TRY,,,,,cleared-MIAG,,,,03515771892,27.03.2023,0847525702,1020,WN#0847525702#20230320#2023#001,18.11.2024,No Matching Requested,18.11.2024,1
5,3142,1000020870,3500007956,,WN,ICA2024000020772,02.04.2024,,-6323.27,,TRY,,,,,cleared-MIAG,,02.04.2024,,03516828256,07.05.2024,0847525702,3381,WN#0847525702#20240402#2024#001,18.11.2024,No Matching Requested,18.11.2024,1
6,3142,1000020870,3500007956,,WN,0023174422,01.07.2023,,-173.9,,TRY,,,,,cleared-MIAG,,,,03516027586,04.07.2023,0847542203,1043,WN#0847542203#20230701#2023#001,18.11.2024,No Matching Requested,18.11.2024,1
7,3142,1000015545,3500003274,,WN,ICA2024000049242,16.08.2024,,-1575.6,,TRY,,,,,cleared-MIAG,,16.08.2024,,03517143360,05.09.2024,0847542203,1030,WN#0847542203#20240816#2024#001,18.11.2024,No Matching Requested,18.11.2024,1
8,3142,1000010080,3500000079,,ST,,30.09.2024,,-123.37,,TRY,,,,,cleared-MIAG,,,,03517245499,15.10.2024,2900001076,6091,ST#2900001076#20240930#2025#002,18.11.2024,No Matching Requested,18.11.2024,1
9,3142,1000010080,3500000079,,ST,,30.09.2024,,-78.01,,TRY,,,,,cleared-MIAG,,,,03517245499,15.10.2024,2900001076,6091,ST#2900001076#20240930#2025#004,18.11.2024,No Matching Requested,18.11.2024,1


In [112]:

# loadfile_df['COMPANY_CODE'] = merged_df['BUKRS']

# # consider MIAG and if blank FI system/merged system

# loadfile_df['SUPPLIER_NO']	= merged_df['Supplier number (Sales Line)'].where(merged_df['Supplier number (Sales Line)'].notna(), merged_df['suppl_no'])

# loadfile_df['MIAG_SUPPLIER_NO'] = merged_df['Supplier number (MIAG)']

# loadfile_df['MIAG_SUPPLIER_NO'] = loadfile_df['MIAG_SUPPLIER_NO'].fillna(loadfile_df['SUPPLIER_NO'].map(doc_to_type))

# loadfile_df['ORDER_NO'] = merged_df['AUFNR']

# loadfile_df['DOC_TYPE'] = merged_df['Document type'].where(merged_df['Document type'].notna(), merged_df['Document_type'])

# #sftp/fi/ic
# loadfile_df['INVOICE_NO'] = merged_df['Invoice number'].where(merged_df['Invoice number'].notna(), merged_df['XBLNR'])

# loadfile_df['INVOICE_NO'] = merged_df['Invoice number'].where(merged_df['Invoice number'].notna(), merged_df['XBLNR'])
# loadfile_df['INVOICE_NO'] = merged_df['Invoice number'].where(merged_df['XBLNR'].notna(), merged_df['RENR'])

# #add renr

# loadfile_df['INVOICE_DATE']	= merged_df['REDAT']

# loadfile_df['DELIVERY_NOTE_NO'] = merged_df['LFSNR']

# #loadfile_df['TOTAL_AMT_DC'] = merged_df['GEBRF']

# loadfile_df['TOTAL_AMT_DC'] = merged_df['Gross amount'].where(merged_df['Gross amount'].notna(), merged_df['GEBRF'])

# loadfile_df['TOTAL_AMT_DC'] = merged_df['GEBRF'].where(merged_df['GEBRF'].notna(), merged_df['Amount_in_local_currency'])

# loadfile_df['TOTAL_VAT_DC']	= merged_df['GSMWF']

# loadfile_df['CURRENCY']	= merged_df['WAERS_x']

# condition = (loadfile_df['DOC_TYPE'] == 'MV')

# loadfile_df['PRE_FINANCE_DATE'] = np.where(condition, merged_df['Value date'], '')

# loadfile_df['GOODS_RECEIPT_NO']	= merged_df['WENUM']

# loadfile_df['GOODS_RECEIPT_DATE'] = merged_df['WEDAT']

# loadfile_df['INVOICE_ENTRY_DATE'] = merged_df['RGDAT'].where(merged_df['RGDAT'].notna(), merged_df['BLDAT'])

# loadfile_df['INVOICE_STATUS'] = merged_df['INVOICE_STATUS']

# loadfile_df['INVOICE_STATUS_INTERNAL'] = merged_df['ABGST'] 

# loadfile_df['NET_DUE_DATE'] = merged_df['NET_DUE_DATE']

# loadfile_df['DEBIT_NOTE_NO'] = merged_df['DEBNOTNO']

# loadfile_df['REMITTANCE_ADVICE_NO'] = np.where(
#     merged_df['INVOICE_STATUS'] == 'CLEARED', 
#     merged_df['Remittance advice number'], 
#     '')

# loadfile_df['CLEARING_DATE'] =  merged_df['Value date'].where(merged_df['Value date'].notna(), merged_df['AUGDT'])

# loadfile_df['DOCUMENT_NO']	= merged_df['BELNR'].where(merged_df['BELNR'].notna(), merged_df['Document number'])

# loadfile_df['STORE_NO'] =  merged_df['store_or_dc']

# loadfile_df['ARKTX'] =  merged_df['ARKTX_temp']

# current_date = datetime.date.today()
# formatted_current_date = current_date.strftime("%d.%m.%Y")
# loadfile_df['MATCHING_DATE'] = formatted_current_date
# loadfile_df['MATCH_STATUS'] = 'No Matching Requested'
# loadfile_df['SYNC_DATE']	= formatted_current_date
# loadfile_df['SYNC_STATUS'] = '1'
# loadfile_df= loadfile_df.fillna('')

In [113]:
loadfile_df['DOCUMENT_NO'] = loadfile_df['DOCUMENT_NO'].replace('0000000000', '')

In [131]:
len(loadfile_df[loadfile_df['DEBIT_NOTE_NO']!=''])

37

In [128]:
loadfile_df['DEBIT_NOTE_NO'] = loadfile_df['DEBIT_NOTE_NO'].apply(lambda x: str(x).strip() if str(x).strip() else '')


In [129]:
loadfile_df.dtypes

COMPANY_CODE                int64
SUPPLIER_NO                object
MIAG_SUPPLIER_NO           object
ORDER_NO                   object
DOC_TYPE                   object
INVOICE_NO                 object
INVOICE_DATE               object
DELIVERY_NOTE_NO           object
TOTAL_AMT_DC               object
TOTAL_VAT_DC               object
CURRENCY                   object
PRE_FINANCE_DATE           object
GOODS_RECEIPT_NO           object
GOODS_RECEIPT_DATE         object
INVOICE_ENTRY_DATE         object
INVOICE_STATUS             object
INVOICE_STATUS_INTERNAL    object
NET_DUE_DATE               object
DEBIT_NOTE_NO              object
REMITTANCE_ADVICE_NO       object
CLEARING_DATE              object
DOCUMENT_NO                object
STORE_NO                   object
ARKTX                      object
MATCHING_DATE              object
MATCH_STATUS               object
SYNC_DATE                  object
SYNC_STATUS                object
dtype: object

In [130]:
len(loadfile_df[loadfile_df['DEBIT_NOTE_NO']==''])

9210

In [132]:
current_datetime = datetime.datetime.now().strftime("%Y%m%d")
        
loadfile_df.to_csv(
            r"C:\Users\sappidi.reddy\Downloads\Sample MIAG 3\load.360.35." + current_datetime + ".001_test_internal.csv", sep=',', index=False)

loadfile_df.to_csv(
            r"C:\Users\sappidi.reddy\Downloads\Sample MIAG 3\load.360.35." + current_datetime + ".001_internal.csv", sep=';', index=False)

## Rough Work

In [ ]:
sftp_df_filtered = sftp_df[sftp_df['Invoice number'].notna() & (sftp_df['Invoice number'] != '')]
sftp_df_empty_invs = sftp_df[sftp_df['Invoice number'].isna() | (sftp_df['Invoice number'] == '')]

# Now, find the Invoice numbers in sftp_df that are not in the RENR column of df_ic
result = sftp_df_filtered[~sftp_df_filtered['Invoice number'].isin(df_ic['RENR'])]

# Extract the required columns: Invoice number and Document number
result_list = result[['Invoice number', 'Document number']]

In [ ]:
len(sftp_df_filtered)

In [ ]:
len(sftp_df_empty_invs)

In [ ]:
len(result_list)

In [ ]:
result_list

In [ ]:
result_list_inv = result_list['Invoice number']

In [ ]:
result_list_inv

In [ ]:
inv_no_column_list = result_list['Invoice number'].tolist()

# Joining the list elements with commas and enclosing each in single quotes
formatted_list = ', '.join([f"'{item}'" for item in inv_no_column_list])
print(len(inv_no_column_list))

print(formatted_list)

In [ ]:
merged_df[['BLDAT', 'RGDAT']]

In [ ]:
pd.set_option('display.max_rows', None)

In [ ]:
df_ic.dtypes